In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

file_path = '../drugs.csv'
df = pd.read_csv(file_path)

# Identificar las columnas cualitativas
qualitative_predictors = df.select_dtypes(include=['object']).columns
# Objeto LabelEncoder
le = LabelEncoder()
# Aplicar el codificador a cada columna cualitativa
for col in qualitative_predictors:
    # Transformar las categorías de texto en números
    df[col] = le.fit_transform(df[col])

df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,4
1,47,1,1,0,13.093,2
2,47,1,1,0,10.114,2
3,28,0,2,0,7.798,3
4,61,0,1,0,18.043,4


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import classification_report

# Dividir el conjunto de datos en entrenamiento y prueba
X = df.drop('Drug', axis=1)
y = df['Drug']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Crear un diccionario para almacenar los reportes
reportes = {}

# Definir los criterios y las profundidades a probar
criterios = ['gini', 'entropy']
profundidades = [3, 5, 7, None]  # None significa sin límite de profundidad

# Entrenar y evaluar el modelo para cada combinación de criterio y profundidad
for criterio in criterios:
    for profundidad in profundidades:
        # Crear y entrenar el modelo
        clf = DecisionTreeClassifier(criterion=criterio, max_depth=profundidad, random_state=1)
        clf.fit(X_train, y_train)
        # Predecir con el modelo entrenado
        y_pred = clf.predict(X_test) 
        # Generar el reporte de clasificación
        reporte = classification_report(y_test, y_pred)
        reportes[(criterio, profundidad)] = reporte

# Mostrar los reportes de clasificación uno por uno
for key, value in reportes.items():
    print(f"Reporte para criterio: {key[0]}, profundidad: {key[1]}")
    print(value)
    print("\n")



Reporte para criterio: gini, profundidad: 3
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       1.00      0.67      0.80         6
           2       0.00      0.00      0.00         4
           3       0.83      1.00      0.90        19
           4       1.00      1.00      1.00        27

    accuracy                           0.90        60
   macro avg       0.70      0.73      0.70        60
weighted avg       0.86      0.90      0.87        60



Reporte para criterio: gini, profundidad: 5
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       1.00      0.67      0.80         6
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        27

    accuracy                           0.97        60
   macro avg       0.93      0.93      0.

para cada criterio que tiene orfundidad mayor a 5 tenemos una precisión del 97%, por lo que no hay una grab variabilidad en los resultados.

In [3]:
# Entrenar y visualizar las reglas del árbol de decisión con profundidad 5 para el criterio 'gini'
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=42)
clf_gini.fit(X_train, y_train)
rules_gini = export_text(clf_gini, feature_names=list(X.columns))
print("Reglas del árbol de decisión (criterio: gini, profundidad: 5):")
print(rules_gini)

# Entrenar y visualizar las reglas del árbol de decisión con profundidad 5 para el criterio 'entropy'
clf_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42)
clf_entropy.fit(X_train, y_train)
rules_entropy = export_text(clf_entropy, feature_names=list(X.columns))
print("Reglas del árbol de decisión (criterio: entropy, profundidad: 5):")
print(rules_entropy)

Reglas del árbol de decisión (criterio: gini, profundidad: 5):
|--- Na_to_K <= 14.84
|   |--- BP <= 0.50
|   |   |--- Age <= 53.00
|   |   |   |--- class: 0
|   |   |--- Age >  53.00
|   |   |   |--- class: 1
|   |--- BP >  0.50
|   |   |--- BP <= 1.50
|   |   |   |--- Cholesterol <= 0.50
|   |   |   |   |--- class: 2
|   |   |   |--- Cholesterol >  0.50
|   |   |   |   |--- class: 3
|   |   |--- BP >  1.50
|   |   |   |--- class: 3
|--- Na_to_K >  14.84
|   |--- class: 4

Reglas del árbol de decisión (criterio: entropy, profundidad: 5):
|--- Na_to_K <= 14.84
|   |--- BP <= 0.50
|   |   |--- Age <= 53.00
|   |   |   |--- class: 0
|   |   |--- Age >  53.00
|   |   |   |--- class: 1
|   |--- BP >  0.50
|   |   |--- BP <= 1.50
|   |   |   |--- Cholesterol <= 0.50
|   |   |   |   |--- class: 2
|   |   |   |--- Cholesterol >  0.50
|   |   |   |   |--- class: 3
|   |   |--- BP >  1.50
|   |   |   |--- class: 3
|--- Na_to_K >  14.84
|   |--- class: 4



Árbol de Decisión (criterio: gini, profundidad: 5)
Nodo Raíz:

Condición: Na_to_K <= 14.84
Si Na_to_K es menor o igual a 14.84, seguimos a la izquierda.
Si Na_to_K es mayor a 14.84, seguimos a la derecha y clasificamos como clase 4.
Rama Izquierda (Na_to_K <= 14.84):

Condición: BP <= 0.50
Si BP es menor o igual a 0.50, seguimos a la izquierda.
Si BP es mayor a 0.50, seguimos a la derecha.
Sub-rama Izquierda (BP <= 0.50):

Condición: Age <= 53.00
Si Age es menor o igual a 53.00, clasificamos como clase 0.
Si Age es mayor a 53.00, clasificamos como clase 1.
Sub-rama Derecha (BP > 0.50):

Condición: BP <= 1.50
Si BP es menor o igual a 1.50, seguimos a la izquierda.
Si BP es mayor a 1.50, clasificamos como clase 3.
Sub-sub-rama Izquierda (BP <= 1.50):

Condición: Cholesterol <= 0.50
Si Cholesterol es menor o igual a 0.50, clasificamos como clase 2.
Si Cholesterol es mayor a 0.50, clasificamos como clase 3.


In [4]:
# Datos del paciente específico
patient_data = {
    'Age': 50,
    'Sex': 'F',
    'BP': 'HIGH',
    'Cholesterol': 'NORMAL',
    'Na_to_K': 15.302
}

# Crear un DataFrame para el paciente y codificar las variables cualitativas
patient_df = pd.DataFrame([patient_data])
# Asegurarnos de codificar solo las columnas presentes en patient_df
for col in patient_df.select_dtypes(include=['object']).columns:
    patient_df[col] = le.fit_transform(patient_df[col])

# Predecir la clase del medicamento usando el modelo con criterio 'gini' y profundidad 5
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=1)
clf_gini.fit(X_train, y_train)
predicted_class_gini = clf_gini.predict(patient_df)[0]
print(f"Predicción para el paciente (criterio: gini, profundidad: 5): Clase {predicted_class_gini}")

# Predecir la clase del medicamento usando el modelo con criterio 'entropy' y profundidad 5
clf_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=1)
clf_entropy.fit(X_train, y_train)
predicted_class_entropy = clf_entropy.predict(patient_df)[0]
print(f"Predicción para el paciente (criterio: entropy, profundidad: 5): Clase {predicted_class_entropy}")


Predicción para el paciente (criterio: gini, profundidad: 5): Clase 4
Predicción para el paciente (criterio: entropy, profundidad: 5): Clase 4
